In [18]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Load the data from CSV file
data = pd.read_csv('resisData.csv')

# Convert necessary columns to numeric
columns_to_convert = ['air_gram', 'energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram', 'serat_gram', 'abu_gram', 'kalsium_mg', 'fosfor_mg', 'zatbesi_mg', 'natrium_mg', 'kalium_mg', 'tembaga_mg', 'seng_mg', 'retinol_mcg', 'bkaroten_mcg', 'karotentotal_mcg', 'thiamin_mg', 'riboflavin_mg', 'niasin_mg', 'vitc_mg']
data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Drop rows with any missing values
data.dropna(inplace=True)

def nutrition_need(umur, tb, bb, aktifitas, kondisi, waktu_makan):
    aktivitas_factor = {"Bed Rest": 1.1, "bergerak terbatas": 1.2, "bisa jalan": 1.3}
    if aktifitas not in aktivitas_factor: return False
    aktifitas = aktivitas_factor[aktifitas]
    
    BMR_perempuan = 655 + (9.6 * bb) + (1.8 * tb) - (4.7 * umur)
    total_kalori = BMR_perempuan * aktifitas
    karbohidrat = (0.6 * total_kalori) / 4
    lemak = (0.25 * total_kalori) / 9
    protein = (0.15 * total_kalori) / 4
    
    batas_atas = {'energi_kal': total_kalori, 'karbohidrat_gram': karbohidrat, 'lemak_gram': lemak, 'protein_gram': protein}
    age_based_limits = [(11, 12, {'serat_gram': 27, 'air_gram': 1850, 'vitc_mg': 50, 'retinol_mcg': 600, 'kalsium_mg': 1200, 'fosfor_mg': 1250, 'zatbesi_mg': 8, 'kalium_mg': 4400, 'natrium_mg': 1400, 'tembaga_mg': 700}),
                        (13, 15, {'serat_gram': 27, 'air_gram': 2100, 'vitc_mg': 65, 'retinol_mcg': 600, 'kalsium_mg': 1200, 'fosfor_mg': 1250, 'zatbesi_mg': 15, 'kalium_mg': 4800, 'natrium_mg': 1500, 'tembaga_mg': 795}),
                        (16, 18, {'serat_gram': 29, 'air_gram': 2150, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200, 'fosfor_mg': 1250, 'zatbesi_mg': 15, 'kalium_mg': 5000, 'natrium_mg': 1600, 'tembaga_mg': 890}),
                        (19, 29, {'serat_gram': 32, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1000, 'fosfor_mg': 700, 'zatbesi_mg': 18, 'kalium_mg': 4700, 'natrium_mg': 1500, 'tembaga_mg': 900}),
                        (30, 49, {'serat_gram': 30, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1000, 'fosfor_mg': 700, 'zatbesi_mg': 18, 'kalium_mg': 4700, 'natrium_mg': 1500, 'tembaga_mg': 900}),
                        (50, 64, {'serat_gram': 25, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200, 'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1400, 'tembaga_mg': 900}),
                        (65, 80, {'serat_gram': 22, 'air_gram': 1550, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200, 'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1200, 'tembaga_mg': 900}),
                        (80, float('inf'), {'serat_gram': 20, 'air_gram': 1400, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200, 'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1000, 'tembaga_mg': 900})]
    
    for min_age, max_age, limits in age_based_limits:
        if min_age <= umur <= max_age: batas_atas.update(limits); break
    
    kondisi_based_adjustments = {'hamil_trim_1': {'energi_kal': 180, 'karbohidrat_gram': 25, 'lemak_gram': 2.3, 'protein_gram': 1, 'serat_gram': 3, 'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'tembaga_mg': 100},
                                 'hamil_trim_2': {'energi_kal': 300, 'karbohidrat_gram': 40, 'lemak_gram': 2.3, 'protein_gram': 10, 'serat_gram': 4, 'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'tembaga_mg': 100},
                                 'hamil_trim_3': {'energi_kal': 300, 'karbohidrat_gram': 40, 'lemak_gram': 2.3, 'protein_gram': 30, 'serat_gram': 4, 'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'tembaga_mg': 100},
                                 'menyusui_6_awal': {'energi_kal': 330, 'karbohidrat_gram': 45, 'lemak_gram': 2.2, 'protein_gram': 20, 'serat_gram': 5, 'air_gram': 800, 'vitc_mg': 45, 'retinol_mcg': 350, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'kalium_mg': 400, 'tembaga_mg': 100},
                                 'menyusui_6_kedua': {'energi_kal': 400, 'karbohidrat_gram': 55, 'lemak_gram': 2.4, 'protein_gram': 15, 'serat_gram': 5, 'air_gram': 700, 'vitc_mg': 25, 'retinol_mcg': 350, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'kalium_mg': 400, 'tembaga_mg': 100}}
    
    if kondisi in kondisi_based_adjustments:
        adjustments = kondisi_based_adjustments[kondisi]
        for key, value in adjustments.items(): batas_atas[key] += value

    for key in batas_atas: batas_atas[key] /= waktu_makan

    return batas_atas

def get_food_nutrients(food_list, data):
    nutrients = data[data['nama_bahan'].isin(food_list)]
    return nutrients

def recommend_food(nutrients, data, k=5):
    nutrient_values = nutrients[columns_to_convert].values
    all_nutrient_values = data[columns_to_convert].values
    
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(all_nutrient_values)
    
    distances, indices = knn.kneighbors(nutrient_values)
    recommendations = data.iloc[indices[0]].drop_duplicates(subset='jenis_pangan', keep='first')

    return recommendations

def isi_piringku_recommendations(recommendations):
    piringku = {"makanan_pokok": [], "lauk_pauk": [], "buah": [], "sayuran": []}

    for _, row in recommendations.iterrows():
        if row['jenis_pangan'] in ["serealia", "umbi berpati"]:
            piringku["makanan_pokok"].append(row)
        elif row['jenis_pangan'] in ["ikan kerang udang", "daging unggas", "kacang biji bean"]:
            piringku["lauk_pauk"].append(row)
        elif row['jenis_pangan'] == "buah":
            piringku["buah"].append(row)
        elif row['jenis_pangan'] == "sayuran":
            piringku["sayuran"].append(row)
    
    return piringku

# Input dari user
umur = 25
tb = 160
bb = 60
aktifitas = "bisa jalan"
kondisi = "normal"
waktu_makan = 3
food_list = ["nasi", "apel segar", "sop buntut masakan"]

# Hitung kebutuhan nutrisi
nutritional_needs = nutrition_need(umur, tb, bb, aktifitas, kondisi, waktu_makan)

# Dapatkan nutrisi dari makanan yang dimakan
nutrients = get_food_nutrients(food_list, data)

# Rekomendasikan makanan
recommendations = recommend_food(nutrients, data)

# Susun rekomendasi isi piringku
piringku = isi_piringku_recommendations(recommendations)

# Cetak rekomendasi
print(piringku)


{'makanan_pokok': [id                                                                381
kode                                                            AP092
nama_bahan                                                 nasi rames
sumber                                                      KZGMS1993
air_gram                                                         65.8
energi_kal                                                      155.0
protein_gram                                                     10.3
lemak_gram                                                        4.2
karbohidrat_gram                                                 19.1
serat_gram                                                        0.0
abu_gram                                                          0.6
kalsium_mg                                                      239.0
fosfor_mg                                                        62.0
zatbesi_mg                                                        1.8
n

In [46]:
import pandas as pd

# Load data
data = pd.read_csv('resisData.csv')

# Function to recommend food for a plate
def rekomendasi_makanan(data):
    max_calories = 700
    batas_pokok = (2/6) * max_calories
    batas_lauk = (1/6) * max_calories
    batas_buah = (1/6) * max_calories
    batas_sayur = (2/6) * max_calories
    
    while True:
        # Filtering data based on food categories
        makanan_pokok = data[data['jenis_pangan'].isin(['serealia', 'umbi berpati'])]
        lauk_pauk = data[data['jenis_pangan'].isin(['ikan kerang udang', 'daging unggas', 'kacang biji bean'])]
        buah = data[data['jenis_pangan'] == 'buah']
        sayuran = data[data['jenis_pangan'] == 'sayuran']

        # Randomly select one item from each category
        makanan_pokok_selected = makanan_pokok.sample(n=1)
        lauk_pauk_selected = lauk_pauk.sample(n=1)
        buah_selected = buah.sample(n=1)
        sayuran_selected = sayuran.sample(n=1)

        # Calculate calories for each category
        kalori_pokok = makanan_pokok_selected.iloc[0]['energi_kal']
        kalori_lauk = lauk_pauk_selected.iloc[0]['energi_kal']
        kalori_buah = buah_selected.iloc[0]['energi_kal']
        kalori_sayur = sayuran_selected.iloc[0]['energi_kal']

        # Calculate total calories
        total_calories = kalori_pokok + kalori_lauk + kalori_buah + kalori_sayur
        
        # Check if the calories for each category are within the allowed limit
        if (kalori_pokok <= batas_pokok and
            kalori_lauk <= batas_lauk and
            kalori_buah <= batas_buah and
            kalori_sayur <= batas_sayur and
            total_calories <= max_calories):
            plate = {
                'makanan pokok': makanan_pokok_selected.iloc[0]['nama_bahan'],
                'lauk pauk': lauk_pauk_selected.iloc[0]['nama_bahan'],
                'buah': buah_selected.iloc[0]['nama_bahan'],
                'sayur-sayuran': sayuran_selected.iloc[0]['nama_bahan'],
                'total kalori': total_calories
            }
            return plate

# Generate recommendations
rekomendasi_piring = []
for i in range(3):  # Generate 3 plates
    rekomendasi_piring.append(rekomendasi_makanan(data))

# Output recommendations
for idx, piring in enumerate(rekomendasi_piring, 1):
    print(f'Piring {idx}')
    for kategori, makanan in piring.items():
        print(f'{kategori} => {makanan}')

Piring 1
makanan pokok => Nasi coklat
lauk pauk => kuro segar
buah => pisang tujuh bulan
sayur-sayuran => bunga pepaya segar
total kalori => 338.0
Piring 2
makanan pokok => nasi gemuk
lauk pauk => ampas tahu mentah
buah => alpukat segar
sayur-sayuran => daun gunda bali segar
total kalori => 394.0
Piring 3
makanan pokok => jagung gerontol
lauk pauk => jukku pallu kaloa masakan
buah => pisang lampung segar
sayur-sayuran => selada rebus
total kalori => 289.0


In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

# Parameters
algo = "auto"
met = "cosine"

# Load data
data = pd.read_csv('resisData.csv')

# Drop unnecessary columns
data = data.drop(['id', 'kode', 'sumber', 'gambar', 'satuan'], axis=1)
data['jenis_pangan_encoded'] = pd.factorize(data['jenis_pangan'])[0]

# Define numeric columns
numeric_cols = ["jenis_pangan_encoded", "air_gram", "energi_kal", "protein_gram", "lemak_gram", "karbohidrat_gram",
                "serat_gram", "natrium_mg", "kalium_mg", "bdd"]

# Replace commas with dots in numeric columns and convert to float
data[numeric_cols] = data[numeric_cols].replace({',': '.'}, regex=True)
data[numeric_cols] = data[numeric_cols].astype(float)

# Normalize data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data[numeric_cols])

# Using Nearest Neighbors
model = NearestNeighbors(n_neighbors=6, algorithm=algo, metric=met)
model.fit(data_normalized)

# Function to get food recommendations
def get_recommendations(food_names, allergy_list):
    # Find indices of the given food names
    food_indices = [data[data['nama_bahan'] == food_name].index[0] for food_name in food_names]
    
    # Find nearest neighbors
    distances, indices = model.kneighbors(data_normalized[food_indices])
    
    # Get food names and distances
    recommendations = []
    jarak = []
    for idx_list, dist_list in zip(indices, distances):
        for idx, dist in zip(idx_list, dist_list):
            if idx not in food_indices and data.iloc[idx]['nama_bahan'] not in allergy_list:
                recommendations.append(data.iloc[idx]['nama_bahan'])
                jarak.append(dist)
    
    return recommendations[:10], jarak[:10]

# Function to recommend food for a plate
def rekomendasi_piring(user_foods, n_recommendations=3):
    rekomendasi = []
    for food in user_foods:
        recs, _ = get_recommendations([food], [])
        rekomendasi.append(recs[:n_recommendations])
    
    plates = []
    for i in range(n_recommendations):
        plate = []
        for rec_list in rekomendasi:
            plate.append(rec_list[i])
        plates.append(plate)
    
    return plates

# Contoh penggunaan
user_foods = ['kentang segar', 'apel segar', 'Ikan salmon', 'bayam rebus']
recommendations = rekomendasi_piring(user_foods)

# Output recommendations
for idx, plate in enumerate(recommendations, 1):
    print(f'Piring {idx}')
    for food in plate:
        print(f' - {food}')


Piring 1
 - bengkuang segar
 - jambu air segar
 - Ikan tuna
 - caisin segar
Piring 2
 - suweg talas segar
 - jambu monyet segar
 - ikan mujahir pepes
 - tomat air (sari) segar
Piring 3
 - ubi jalar kuning kukus
 - apel malang segar
 - ikan sidat segar
 - kangkung segar


In [18]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Data makanan dan nutrisi (contoh sederhana)
data = {
    'food': ['nasi', 'apel segar', 'sop buntut', 'bayam rebus', 'nasi uduk', 'apel malang', 'sop daging', 'kangkung segar'],
    'category': ['serealia', 'buah', 'daging unggas', 'sayuran', 'serealia', 'buah', 'daging unggas', 'sayuran'],
    'calories': [200, 50, 150, 30, 220, 55, 160, 35]
}
df = pd.DataFrame(data)

# Contoh data input user
user_data = {
    'age': 30,
    'height': 160,
    'weight': 60,
    'activity': 'moderate',
    'condition': 'pregnant',
    'meals_per_day': 3,
    'allergies': ['daging unggas']
}

# Fungsi untuk menghitung kebutuhan nutrisi (contoh sederhana)
def calculate_nutrition_need(user_data):
    base_calories = 1800  # contoh sederhana, bisa disesuaikan dengan rumus yang lebih kompleks
    if user_data['activity'] == 'moderate':
        base_calories += 200
    if user_data['condition'] == 'pregnant':
        base_calories += 300
    return base_calories

# Fungsi untuk mendapatkan rekomendasi piring makanan
def get_food_recommendations(user_data, df):
    nutrition_need = calculate_nutrition_need(user_data)
    per_meal_calories = nutrition_need / user_data['meals_per_day']
    
    # Filter makanan berdasarkan alergi
    filtered_df = df[~df['category'].isin(user_data['allergies'])]
    
    # KNN untuk rekomendasi makanan
    knn = NearestNeighbors(n_neighbors=3, algorithm='auto').fit(filtered_df[['calories']])
    
    # Membangun piring makanan
    plates = []
    for i in range(user_data['meals_per_day']):
        # Sample makanan untuk setiap kategori
        try:
            staples = filtered_df[filtered_df['category'] == 'serealia'].sample(1)
        except ValueError:
            staples = df[df['category'] == 'serealia'].sample(1)
        
        try:
            sides = filtered_df[filtered_df['category'] == 'daging unggas'].sample(1)
        except ValueError:
            sides = df[df['category'] == 'daging unggas'].sample(1)
        
        try:
            fruits = filtered_df[filtered_df['category'] == 'buah'].sample(1)
        except ValueError:
            fruits = df[df['category'] == 'buah'].sample(1)
        
        try:
            veggies = filtered_df[filtered_df['category'] == 'sayuran'].sample(1)
        except ValueError:
            veggies = df[df['category'] == 'sayuran'].sample(1)
        
        plate = pd.concat([staples, sides, fruits, veggies])
        plates.append(plate['food'].tolist())
    
    return plates

# Mendapatkan rekomendasi makanan
recommended_plates = get_food_recommendations(user_data, df)

# Output rekomendasi piring makanan
for i, plate in enumerate(recommended_plates, 1):
    print(f"Plate {i}: {', '.join(plate)}")


Plate 1: nasi, sop daging, apel segar, bayam rebus
Plate 2: nasi, sop daging, apel segar, kangkung segar
Plate 3: nasi, sop daging, apel segar, bayam rebus
